In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
import torch
from torch import cuda
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json

In [11]:

# Load the pre-trained T5 model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("mxmax/Chinese_Chat_T5_Base")
model = AutoModelForSeq2SeqLM.from_pretrained("mxmax/Chinese_Chat_T5_Base") 
device = 'cuda' if cuda.is_available() else 'cpu'
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [28]:

def postprocess(text):
    return text.replace(".", "").replace('</>','')

def answer_fn(text, top_k=50):
    encoding = tokenizer(text=[text], truncation=True, padding=True, max_length=256, return_tensors="pt").to(device) 
    out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_length=512,temperature=0.5,do_sample=True,repetition_penalty=3.0 ,top_k=top_k)
    result = tokenizer.batch_decode(out["sequences"], skip_special_tokens=True)
    return postprocess(result[0])

x1 = """世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得”，就需要各方面密切配合、（凭空捏造|高头大马|通力合作|同舟共济|和衷共济|蓬头垢面|紧锣密鼓）。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、(叫苦连天|量体裁衣|金榜题名|百战不殆|知彼知己|风流才子)、
灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。请从括号里选择出合适的成语"""
# y1 = ["高头大马", "叫苦连天"]

result=answer_fn(x1, top_k=50)
print("model result:",result)
print('*'*100)


model result: 冠冕堂皇、无所不在。
****************************************************************************************************


In [29]:
# Preprocess the inputs and outputs
def preprocess_data(data):
    input_texts = []
    labels = []

    for example in data:
        example = json.loads(example)

        input_text = example['content']
        ground_truth = example['groundTruth']
        candidates = example['candidates']
        
        for i, idiom in enumerate(ground_truth):
            candidates_str = '|'.join([c for c in candidates[i] if c != idiom])
            input_text = input_text.replace('#idiom#', "(" + candidates_str + ")", 1)
            input_text += "请从括号里选择出合适的成语"
        input_texts.append(input_text)
        labels.append('、'.join(ground_truth))
        
    print(f"input_texts: {input_texts[0]}\n")
    inputs = tokenizer(text=input_texts, truncation=True, padding=True, max_length=256, return_tensors="pt").to(device)
    
    print(f"labels: {labels[0]}\n")
    labels = tokenizer(text=labels, truncation=True, padding=True, max_length=256, return_tensors="pt").to(device)
    return inputs, labels


In [30]:
# Load the Chinese Idioms dataset
train_data_file = './ChID/train_data.txt'
val_data_file = './ChID/dev_data.txt'


with open(train_data_file) as f:
    train_data = f.readlines()

with open(val_data_file) as f:
    val_data = f.readlines()

train_inputs, train_labels = preprocess_data(train_data)
val_inputs, val_labels = preprocess_data(val_data)

input_texts: 由实力派演员刘威饰演的清华第三任校长蒋南翔，是我国著名的青年运动家和教育家，他跟清华终身校长梅贻琦一样，都是由清华人自己培养出来的校长。历史上的蒋南翔是著名的“一二九”学生救亡运动的领导人之一，他在清华校长之位14年期间，不但很好的继承了清华建校之初的优秀传统与理念，而且更加的(意气风发|街谈巷议|人才辈出|一脉相传|后继有人|腥风血雨)，他把清华的教师队伍扩大了将近5倍，将清华本科人数破万，为新中国培养了大量的有用人才。在《天行健》中饰演蒋南翔的刘威是观众所熟悉的著名实力派演员，早在1987年刘威就在《关东大侠》中饰演豪爽仗义的关云天一角而获得了金鸡奖最佳男主角的提名，后来更是因在《唐明皇》中精湛的表演而一举夺得金鹰奖最佳男演员奖。此次《天行健》选定刘威来出演正是看中了他(落落大方|八仙过海|彬彬有礼|史无前例|盛气凌人|好自为之)的表演方式和对人物深入内心的刻画。至此，《天行健》中涉及的三位清华校长的人选都已经曝光，(不拘小节|风流潇洒|无病呻吟|言谈举止|壮志凌云|关门闭户)的第一任校长赵文?、稳重坚毅的第二任校长孙逊、亲切务实的第三任校长刘威，再加上梁思成、林徽因、朱自清、闻一多等一批“大师”的加盟，相信作为清华百年校庆重点项目之一的《天行健》一定会带领观众重温那段不能抹去的历史。请从括号里选择出合适的成语请从括号里选择出合适的成语请从括号里选择出合适的成语

labels: 发扬光大、平易近人、温文尔雅

input_texts: 另据了解，北京一个对垃圾短信(人人自危|恨入骨髓|不胜枚举|嗤之以鼻|走马看花|不屑一顾)的老人，利用该软件总共呼死了近2000个号码。20分钟呼上万号码记者昨天在百度里输入“呼死你软件”，出现了7000多个相关网页，随机登录几个网站，发现软件均需花钱购买，价格从200元至500元不等。请从括号里选择出合适的成语

labels: 深恶痛绝

